In [44]:
import pandas as pd
from functools import reduce
import statsmodels.api as sm
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split

In [45]:
df = pd.read_csv("IL_public_data.csv").iloc[:, 1:]


In [46]:
#drop any column that has almost 40% missing values
res2 = df.columns[df.isnull().sum() > 47]
df.drop(res2, inplace=True, axis=1)
df.set_index(['year', 'UNITID'], inplace=True)

In [47]:
df.isna().sum().sum()

1802

In [48]:
# #Getting rid of Governors State, because it is missing graduation rates
# df = df[df["UNITID"] != 145336]

KeyError: 'UNITID'

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 120 entries, (2019, 144005) to (2010, 149772)
Columns: 203 entries, ENRTOT to DOCDEGOT
dtypes: float64(203)
memory usage: 191.4 KB


In [50]:
df.describe()

,ENRTOT,FTE,ENRFT,ENRPT,PCTENRWH,PCTENRBK,PCTENRHS,PCTENRAP,PCTENRAS,PCTENRNH,...,SASCDEG,SBAMACRT,SCERT24,SCERT1,UNDUP,UNDUPUG,FTE12MN,DOCDEGRS,DOCDEGPP,DOCDEGOT
count,120.000000,120.000000,120.00000,120.000000,120.0000,120.000000,120.000000,120.000000,120.000000,120.0,...,96.000000,96.000000,96.000000,96.0,120.000000,120.000000,120.000000,108.000000,108.000000,108.000000
mean,16035.108333,14009.891667,12761.25000,3273.858333,53.4250,18.783333,10.708333,5.316667,5.258333,0.0,...,7.177083,33.416667,0.135417,0.0,18411.358333,13333.075000,15164.166667,124.685185,124.138889,1.824074
std,11779.025065,11311.346914,11071.20192,1282.241763,20.0049,18.045779,7.981487,5.404610,5.363267,0.0,...,24.396719,37.265383,0.828693,0.0,12196.548407,8435.054202,12844.086191,228.170483,185.485544,5.859458
min,2964.000000,2258.000000,1810.00000,1154.000000,5.0000,5.000000,3.000000,1.000000,1.000000,0.0,...,0.000000,0.000000,0.000000,0.0,3593.000000,2438.000000,2260.000000,0.000000,0.000000,0.000000
25%,7273.000000,5533.000000,4326.25000,2334.500000,41.0000,9.000000,6.000000,2.000000,2.000000,0.0,...,0.000000,9.000000,0.000000,0.0,8967.500000,6749.500000,5798.750000,1.000000,0.000000,0.000000
50%,12569.500000,11159.500000,10127.00000,3047.500000,60.0000,13.000000,8.000000,3.000000,2.500000,0.0,...,0.000000,26.000000,0.000000,0.0,14822.500000,11891.000000,11345.500000,10.000000,37.000000,0.000000
75%,20719.500000,19132.750000,17923.50000,4143.500000,69.0000,17.000000,11.250000,6.500000,6.500000,0.0,...,0.000000,44.000000,0.000000,0.0,23300.750000,18317.250000,18871.500000,137.750000,168.750000,0.000000
max,51605.000000,46818.000000,44022.00000,7583.000000,82.0000,79.000000,36.000000,19.000000,19.000000,0.0,...,102.000000,196.000000,7.000000,0.0,54985.000000,35883.000000,53117.000000,869.000000,726.000000,42.000000


In [51]:
nan_values = df.isna().sum()
nan_columns = nan_values.any()
columns_with_nan = df.columns[nan_columns].tolist()
print(columns_with_nan)                                    

[['ENRTOT', 'FTE', 'ENRFT', 'ENRPT', 'PCTENRWH', 'PCTENRBK', 'PCTENRHS', 'PCTENRAP', 'PCTENRAS', 'PCTENRNH', 'PCTENRAN', 'PCTENR2M', 'PCTENRUN', 'PCTENRNR', 'PCTENRW', 'PCTFT1ST', 'EFUG', 'EFUG1ST', 'EFUGTRN', 'EFUGCNT', 'EFUGNDG', 'EFUGFT', 'EFUG1SFT', 'EFUGTRFT', 'EFUGCNFT', 'EFUGNDFT', 'EFUGPT', 'EFUG1SPT', 'EFUGTRPT', 'EFUGCNPT', 'EFUGNDPT', 'PCUENRWH', 'PCUENRBK', 'PCUENRHS', 'PCUENRAP', 'PCUENRAS', 'PCUENRNH', 'PCUENRAN', 'PCUENR2M', 'PCUENRUN', 'PCUENRNR', 'PCUENRW', 'EFGRAD', 'EFGRADFT', 'EFGRADPT', 'PCGENRWH', 'PCGENRBK', 'PCGENRHS', 'PCGENRAP', 'PCGENRAS', 'PCGENRNH', 'PCGENRAN', 'PCGENR2M', 'PCGENRUN', 'PCGENRNR', 'PCGENRW', 'DVEF01', 'DVEF02', 'DVEF03', 'DVEF05', 'DVEF06', 'DVEF07', 'DVEF09', 'DVEF10', 'DVEF11', 'DVEF13', 'DVEF14', 'DVEF15', 'DVEF16', 'RMINSTTN', 'RMOUSTTN', 'RMFRGNCN', 'RMUNKNWN', 'RMINSTTP', 'RMOUSTTP', 'RMFRGNCP', 'RMUNKNWP', 'PCTDEEXC', 'PCTDESOM', 'PCTDENON', 'PCUDEEXC', 'PCUDESOM', 'PCUDENON', 'PCGDEEXC', 'PCGDESOM', 'PCGDENON', 'GRRTTOT', 'GRRTM', 'G

In [53]:
df.dropna(axis=1,inplace=True)
df.isna().sum().sum()

0

In [54]:
import statsmodels.api as sm
			
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            #if verbose:
                #print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            #if verbose:
                #print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    included.append('id')
    print('resulting features:')
    print(included)
    
    return included

In [55]:
#set target graduation rates
y = df['F1CORREV']
X = df.drop(columns=['F1CORREV'], axis=1)

In [56]:
stepwise_selection(X,y)

<ipython-input-54-59576973df7c>:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-54-59576973df7c>:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-54-59576973df7c>:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-54-59576973df7c>:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
<ipython-input-54-59576973df7c>:26: 

resulting features:
['F1COREXP', 'PCUENRAS', 'EFUGPT', 'PCGENRNR', 'id']


['F1COREXP', 'PCUENRAS', 'EFUGPT', 'PCGENRNR', 'id']

In [ ]:
X.columns